# GoogLeNet
---

GoogLeNet，是Google团队在2014年提出的一种深度卷积神经网络架构，它在当年的ImageNet大规模视觉识别挑战赛（ILSVRC）中取得了优异的成绩，赢得了冠军。GoogLeNet的设计目标是提高网络的深度和宽度，同时保持计算效率和参数数量在可控范围内。

## 主要特点

1. Inception模块：
    GoogLeNet的核心创新是Inception模块，这是一种多尺度卷积的组合方式。传统的卷积神经网络通常使用固定大小的卷积核（如3x3或5x5），而Inception模块则同时使用多种不同大小的卷积核（如1x1、3x3、5x5）以及池化层，然后将它们的输出在通道维度上拼接在一起。这种设计允许网络在不同尺度上捕捉特征，从而提高了网络的表达能力。

    - 1x1卷积：Inception模块中广泛使用了1x1卷积，这种卷积核可以在不改变空间维度的情况下减少通道数，从而降低计算复杂度。1x1卷积也被称为“瓶颈层”，因为它在保持特征图尺寸不变的同时，减少了特征图的通道数，从而减少了后续计算的负担。

2. 深度和宽度：
    GoogLeNet通过堆叠多个Inception模块来增加网络的深度和宽度。网络的深度指的是网络中层的数量，而宽度指的是每一层中特征图的数量。GoogLeNet通过Inception模块的设计，能够在不显著增加计算复杂度的情况下，有效地增加网络的深度和宽度。

3. 全局平均池化：
    GoogLeNet在网络的最后使用了全局平均池化层（Global Average Pooling），而不是传统的全连接层。全局平均池化层将每个特征图的所有值取平均，得到一个单一的值，然后将这些值组合成一个向量，作为最终的分类输出。这种设计减少了参数数量，降低了过拟合的风险。

4. 辅助分类器：                         *注，李沐未提及此处*
    GoogLeNet在网络的中间层引入了两个辅助分类器，这些分类器在训练过程中提供了额外的梯度信号，帮助网络在早期阶段更好地学习特征。在测试阶段，这些辅助分类器的输出被忽略，只有主分类器的输出被用于最终的预测。



## Inception块
![Inception](https://zh-v2.d2l.ai/_images/inception.svg "Inception")

## 网格结构
1. 输入层：
    - 输入层接收224x224x3的图像数据。

2. 卷积层和池化层：
    - 在进入Inception模块之前，网络首先通过几个卷积层和池化层来提取基本的特征。

3. Inception模块：
    - 网络的主体由多个Inception模块组成，每个模块内部包含多个不同大小的卷积核和池化层，并将它们的输出拼接在一起。

4. 辅助分类器：
    - 在网络的中间部分，引入了两个辅助分类器，它们通过全连接层和softmax层输出分类结果。

5. 全局平均池化层：
    - 在网络的最后，使用全局平均池化层将特征图转换为一个固定长度的向量。

6. 全连接层和softmax层：
    - 最终的全连接层和softmax层用于输出分类结果。

## GoogLeNet架构
![GoogLeNet](https://zh-v2.d2l.ai/_images/inception-full.svg "GoogLeNet")


## 代码简单实现

Inception块

In [1]:
import torch
from torch import nn
from torch.nn import functional as F
from d2l import torch as d2l


class Inception(nn.Module):
    # c1--c4是每条路径的输出通道数
    def __init__(self, in_channels, c1, c2, c3, c4, **kwargs):
        super(Inception, self).__init__(**kwargs)
        # 线路1，单1x1卷积层
        self.p1_1 = nn.Conv2d(in_channels, c1, kernel_size=1)
        # 线路2，1x1卷积层后接3x3卷积层
        self.p2_1 = nn.Conv2d(in_channels, c2[0], kernel_size=1)
        self.p2_2 = nn.Conv2d(c2[0], c2[1], kernel_size=3, padding=1)
        # 线路3，1x1卷积层后接5x5卷积层
        self.p3_1 = nn.Conv2d(in_channels, c3[0], kernel_size=1)
        self.p3_2 = nn.Conv2d(c3[0], c3[1], kernel_size=5, padding=2)
        # 线路4，3x3最大汇聚层后接1x1卷积层
        self.p4_1 = nn.MaxPool2d(kernel_size=3, stride=1, padding=1)
        self.p4_2 = nn.Conv2d(in_channels, c4, kernel_size=1)

    def forward(self, x):
        p1 = F.relu(self.p1_1(x))
        p2 = F.relu(self.p2_2(F.relu(self.p2_1(x))))
        p3 = F.relu(self.p3_2(F.relu(self.p3_1(x))))
        p4 = F.relu(self.p4_2(self.p4_1(x)))
        # 在通道维度上连结输出
        return torch.cat((p1, p2, p3, p4), dim=1)

现在，我们逐一实现GoogLeNet的每个模块。

In [2]:
b1 = nn.Sequential(nn.Conv2d(1, 64, kernel_size=7, stride=2, padding=3),
                   nn.ReLU(),
                   nn.MaxPool2d(kernel_size=3, stride=2, padding=1))

In [3]:
b2 = nn.Sequential(nn.Conv2d(64, 64, kernel_size=1),
                   nn.ReLU(),
                   nn.Conv2d(64, 192, kernel_size=3, padding=1),
                   nn.ReLU(),
                   nn.MaxPool2d(kernel_size=3, stride=2, padding=1))

In [4]:
b3 = nn.Sequential(Inception(192, 64, (96, 128), (16, 32), 32),
                   Inception(256, 128, (128, 192), (32, 96), 64),
                   nn.MaxPool2d(kernel_size=3, stride=2, padding=1))

In [5]:
b4 = nn.Sequential(Inception(480, 192, (96, 208), (16, 48), 64),
                   Inception(512, 160, (112, 224), (24, 64), 64),
                   Inception(512, 128, (128, 256), (24, 64), 64),
                   Inception(512, 112, (144, 288), (32, 64), 64),
                   Inception(528, 256, (160, 320), (32, 128), 128),
                   nn.MaxPool2d(kernel_size=3, stride=2, padding=1))

In [6]:
b5 = nn.Sequential(Inception(832, 256, (160, 320), (32, 128), 128),
                   Inception(832, 384, (192, 384), (48, 128), 128),
                   nn.AdaptiveAvgPool2d((1,1)),
                   nn.Flatten())

net = nn.Sequential(b1, b2, b3, b4, b5, nn.Linear(1024, 10))

GoogLeNet模型的计算复杂，而且不如VGG那样便于修改通道数。 为了使Fashion-MNIST上的训练短小精悍，我们将输入的高和宽从224降到96，这简化了计算。下面演示各个模块输出的形状变化。

In [7]:
X = torch.rand(size=(1, 1, 96, 96))
for layer in net:
    X = layer(X)
    print(layer.__class__.__name__,'output shape:\t', X.shape)

Sequential output shape:	 torch.Size([1, 64, 24, 24])
Sequential output shape:	 torch.Size([1, 192, 12, 12])
Sequential output shape:	 torch.Size([1, 480, 6, 6])
Sequential output shape:	 torch.Size([1, 832, 3, 3])
Sequential output shape:	 torch.Size([1, 1024])
Linear output shape:	 torch.Size([1, 10])
